# Dew Point (VLE)

The isothermal-isobaric two-phase flash is the base for the calculation Vapor-Liquid Equilibria. This calculation is based on the solution of the Rachford-Rice mass balance. 

$$ FO = \sum_{i=1}^c \left( x_i^\beta - x_i^\alpha \right) = \sum_{i=1}^c \frac{z_i (K_i-1)}{1+\psi (K_i-1)} $$


Where,  $K_i = x_i^\beta/x_i^\alpha =\hat{\phi}_i^\alpha /\hat{\phi}_i^\beta  $ represents the equilibrium constant  and $\psi$ the fraction of the phase $\beta$. For bubble and dew points calculations the phase fraction $\psi$ is known beforehand and set to 0 for bubble points (differential size bubble) and to 1 for dew point (differential size liquid drop).

The Rachford-Rice mass balance reduces to the following equations:

###  Dew 

$$ FO = 1 - \sum_{i=1}^c \frac{y_i}{K_i} = 1 - \sum_{i=1}^c x_i = 0 $$

The solution of this calculations includes using accelerated successive substitution (ASS) to update the phase compositions in an inner loop and the quasi-Newton method is used to update pressure or temperature in an outer loop. If slow convergence is detected, the algorithm attempts to solve the following system of equations using equilibrium constants, $K$, as iteration variables. This is done using SciPy's optimization routines.

$$ f_i = \ln K_i + \ln \hat{\phi}_i^v(\underline{y}, T, P) -\ln \hat{\phi}_i^l(\underline{x}, T, P) \quad i = 1,...,c $$
$$ f_{c+1} = \sum_{i=1}^c (y_i-x_i) $$

**note:** these calculations does not check for the stability of the phases.


In this notebook, examples of calculation of dew point properties using Peng-Robinson equation of state are shown. The mixing rule applied is Modified Huron Vidal combined with Redlich Kister polynomial and also with Modified UNIFAC activity coefficient model.

In [1]:
import numpy as np
from phasepy import mixture, component, prsveos, rkeos
from phasepy.equilibrium import dewPx, dewTx

# Binary mixture example

This example uses Peng-Robinson-Stryjek-Vera equation of state. The mixing rule applied is Modified Huron Vidal combined with Redlich-Kister.

In [2]:
ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    ksv=[1.27092923, 0.0440421])

mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 ksv=[0.76429651, 0.04242646])

mix = mixture(mtbe, ethanol)
# or
mix = mtbe + ethanol
C0 = np.array([0.02635196, -0.02855964, 0.01592515])
C1 = np.array([312.575789, 50.1476555, 5.13981131])
mix.rk(C0, C1)
eos = prsveos(mix, mixrule='mhv_rk')

#### Dew point algorithm y, T -> x, P

In [3]:
y = np.array([0.5, 0.5])
T = 350.0
x0 = np.array([0.5, 0.5])
P0 = 1.0
P0 = 10.0
dewPx(x0, P0, y, T, eos, full_output=True)

      T: 350.0
      P: 1.5445973713540904
  error: 1.0031153685474692e-10
   iter: 6
      X: array([0.23512692, 0.76487308])
     v1: 79.85925570828107
 state1: 'Liquid'
      Y: array([0.5, 0.5])
     v2: 18058.942068043507
 state2: 'Vapor'

#### Dew point algorithm y, P -> x, T

In [4]:
y = np.array([0.4, 0.6])
P = 1.0
x0 = np.array([0.1, 0.9])
T0 = 360.0
dewTx(x0, T0, y, P, eos, full_output=True)

      T: 341.7556832715943
      P: 1.0
  error: 1.4876988529977208e-14
   iter: 5
      X: array([0.1357666, 0.8642334])
     v1: 73.24673375255578
 state1: 'Liquid'
      Y: array([0.4, 0.6])
     v2: 27627.203998257115
 state2: 'Vapor'

# Ternary mixture example

This example uses Redlich Kwong equation of state. The mixing rule applied is Modified Huron Vidal combined with Modified UNIFAC.

In [5]:
mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 Ant=[9.16238246, 2541.97883529, -50.40534341],
                 GC={'CH3':3, 'CH3O':1, 'C':1})

ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    Ant=[11.61809279, 3423.0259436, -56.48094263],
                    GC={'CH3':1, 'CH2':1, 'OH(P)':1})

butanol = component(name='n-Butanol', Tc=563.0, Pc=44.14, Zc=0.258, Vc=274.0, w=0.589462,
                    Ant=[9.74673479, 2668.52570016, -116.66189545],
                    GC={'CH3':1, 'CH2':3, 'OH(P)':1})

mix = mixture(mtbe, ethanol)
mix.add_component(butanol)
# or
mix = mtbe + ethanol + butanol

mix.unifac()
eos = rkeos(mix, 'mhv_unifac')

#### Dew point algorithm y, T -> x, P

In [6]:
y = np.array([0.2, 0.5, 0.3])
T = 350.0
x0 = np.array([0.2, 0.5, 0.3])
P0 = 1.0
dewPx(x0, P0, y, T, eos, full_output=True)

      T: 350.0
      P: 4.860774612727455
  error: 1.8918200339612667e-12
   iter: 5
      X: array([0.10835525, 0.35502455, 0.5366202 ])
     v1: 115.51060023270989
 state1: 'Liquid'
      Y: array([0.2, 0.5, 0.3])
     v2: 5398.969154683442
 state2: 'Vapor'

#### Dew point algorithm y, P -> x, T

In [7]:
y = np.array([0.2, 0.5, 0.3])
P = 2.0
x0 = np.array([0.2, 0.5, 0.3])
T0 = 320.0
dewTx(x0, T0, y, P, eos, full_output=True)

      T: 312.2308834003832
      P: 2.0
  error: 2.5730528818679525e-12
   iter: 4
      X: array([0.09352795, 0.33039408, 0.57607797])
     v1: 108.73163446223349
 state1: 'Liquid'
      Y: array([0.2, 0.5, 0.3])
     v2: 12312.022678676827
 state2: 'Vapor'

For further information please also check [official documentation](https://phasepy.readthedocs.io/), or just try:

```function?```